# WaveFrame
This document serves as scratch space for the design of the `WaveFrame`.

In [ ]:
import obsplus

In [ ]:
# First load in a related catalog, stream, and inventory
ds = obsplus.load_dataset('crandall')
st = ds.waveform_client.get_waveforms()
cat = ds.event_client.get_events()
inv = df.station_client.get_stations()

## Create WaveFrame

The simples way to create a `WaveFrame` is from a waveform client and a dataframe. The dataframe, at a minimum, must have the following fields:
    
    ["network", "station", "location", "channel", "starttime", "endtime"]
    
The first four must be strings, and the last two should be either numpy.datetime64 or something that will be converted to such. 

In [ ]:
event_df = obsplus.events_to_df(cat)
wf = obsplus.WaveFrame(stats=event_df, waveforms=st)
# this would also work
wf = obsplus.WaveFrame(stats=event_df, waveforms=ds.waveform_client)

However, there are also other methods for creating WaveFrames from picks, amplitudes, continuous data, etc.

In [ ]:
# waveframe from picks (5 seconds total, 1 before 4 after)
wf = WaveFrame.from_picks(picks=cat, waveforms=st, time_before=1, time_after=4)

In [ ]:
# wavefrom from continuous
wf = WaveFrame.from_continuous(stations=inv, overlap=200, waveforms=st)

## Access dataframes
The dataframes containing the underlying data and metadata can be accessed with the `data` and `stats` attributes, respectively. This is modeled after ObsPy's `Trace` object.

In [ ]:
# get wavefrom dataframe
wf.data

In [ ]:
# get metadata dataframe
wf.stats

## ObsPy interoperability
Converting to and from ObsPy objects is straight-forward:

In [ ]:
# get an obspy stream
st = wf.to_stream()

In [ ]:
# convert a stream to a WaveFrame
wf = WaveFrame.from_stream(st)

## General Behavior
The `WaveFrame` behaves like mix between an ObsPy `Stream` and a pandas `DataFrame`. In order to achieve this the following rules apply:

1. The getitem interface (eg `wf['starttime']`) is used to get a column of the `stats` dataframe.
2. .loc is also used to get/set values in the stats dataframe.
3. Simple operators and numpy functions act on `data` and return a new `WaveFrame`


In [ ]:
# multiply all the waveform values by 5
wf2 = wf * 5
# get max of waveforms
wf2 = wf.max()

In [ ]:
# get rows with station names
wf2 = wf[wf.loc['station'] == "TMU"]

## A peak inside
The `df` attribute allows access to the underlying dataframe. The columns are a multi-index with with the first level containing [`data`, `stats`]. This allows the `WaveFrame` to be serialized with any pandas-supported format which also supports 